In [2]:
import random
POPULATION_SIZE = 10
number_of_parents = 10
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.1
EPOCHS = 500

class Classroom:
    def __init__(self, name, capacity, room_type):
        self.name = name
        self.capacity = capacity
        self.room_type = room_type
    def __str__(self):
        return f"Classroom {self.name} (Capacity: {self.capacity}, Type: {self.room_type})"

class Professor:
    def __init__(self, name, preferred_days=[]):
        self.name = name
        self.preferred_days = preferred_days

class Assistant:
    def __init__(self, name):
        self.name = name

class Lesson:
    def __init__(self, code, teacher, name, is_online, class_type, preferred_days=[], timeslot=None, duration=1, room_preference=[], capacity=None, section=None):
        self.code = code
        self.teacher = teacher
        self.name = name
        self.is_online = is_online
        self.class_type = class_type
        self.preferred_days = preferred_days
        self.timeslot = timeslot
        self.duration = duration
        self.room_preference = room_preference
        self.capacity = capacity
        self.section = section


    def __str__(self):
        return self.code

class ScheduledLesson:
    def __init__(self, lesson, classroom, day, start_timeslot):
        self.lesson = lesson
        self.classroom = classroom
        self.day = day
        self.start_timeslot = start_timeslot

    def __str__(self):
        return f"[{self.lesson.code}, {self.lesson.teacher.name}, {self.classroom.name}, {self.day}, {self.start_timeslot}]"

class Schedule:
    def __init__(self, scheduled_lessons):
        self.scheduled_lessons = scheduled_lessons
        self.fitness = self.calculate_fitness()

    def __str__(self):
        return "\n".join(str(sl) for sl in self.scheduled_lessons)

    def calculate_fitness(self):
        return random.random()  # Placeholder for now


In [3]:

professors = [
    Professor("Dr. Celal Çakıroğlu", preferred_days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), # 0
    Professor("Doç. Dr Enver Vural Yavuz", preferred_days=["Monday", "Tuesday", "Thursday"]), # 1
    Professor("Dr. Ömer Faruk Aydın", preferred_days=["Monday", "Tuesday", "Thursday"]), # 2
    Professor("Dr. Serdar Ulusoy", preferred_days=["Monday", "Tuesday", "Thursday"]), # 3
    Professor("Prof. Dr. F. RACKWITZ", preferred_days=["Tuesday", "Thursday"]), # 4
    Professor("Dr. Öğr. Üyesi Moritz BOTTS", preferred_days=["Tuesday", "Thursday"]), # 5
    Professor("Prof. Dr. Dilek ZAMANTILI NAYIR", preferred_days=["Tuesday", "Thursday"]), # 6
    Professor("M. Kerem Öztürk", preferred_days=["Tuesday", "Thursday"]), # 7
    Professor("Prof. Dr. Müge KLEIN", preferred_days=["Tuesday", "Thursday"]), # 8
    Professor("Dr.Helmut Niesner", preferred_days=["Monday", "Wednesday", "Friday"]), # 9
    Professor("Prof. Dr. Wolfgang KOHN", preferred_days=["Tuesday", "Thursday"]), # 10
    Professor("Dr. Öğr. Üyesi Mehmet Hakan ÖZDEMİR", preferred_days=["Monday", "Tuesday", "Thursday", "Friday"]), # 11
    Professor("Dr. Moritz Martin BOTTS", preferred_days=["Tuesday", "Thursday"]), # 12
    Professor("Prof. Dr. Ela Sibel BAYRAK MEYDANOĞLU", preferred_days=["Tuesday", "Thursday"]), # 13
    Professor("Doç. Dr. Ulrich TAMM", preferred_days=["Monday", "Tuesday", "Thursday"]),   # 14
    Professor("Prof. Dr. Orhan KOCAGÖZ", preferred_days=["Tuesday", "Thursday"]), # 15
    Professor("Prof. Dr. Sibel Özenler", preferred_days=["Tuesday", "Thursday"]), # 16
    Professor("Öğr. Gör. Selahaddin SOYUDOĞRU", preferred_days=["Monday", "Tuesday", "Thursday", "Friday"]), # 17
    Professor("Öğr. Gör. Vahap Sümer Özsüer", preferred_days=["Monday", "Tuesday", "Thursday", "Friday"]), # 18
    Professor("Dr. Öğr. Üyesi Gökhan Habiboğlu", preferred_days=["Monday", "Tuesday", "Thursday", "Friday"]), # 19
    Professor("Dr.-Ing. Sanam Moghaddamnia", preferred_days=["Monday", "Tuesday", "Thursday", "Friday"]), # 20
    Professor("Dr. Öğr. Üyesi Murat Tümer", preferred_days=["Tuesday", "Thursday"]), # 21
    Professor("Prof. Dr. Cem Civelek", preferred_days=["Monday", "Wednesday", "Friday"]),  # 22
    Professor("Doç. Dr. Tanju Yelkenci", preferred_days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), # 23
    Professor("Prof. Dr. Georg Duschl-Graw (FF)", preferred_days=["Tuesday", "Thursday"]), # 24
    Professor("Andrea Schwandt (FF)", preferred_days=["Tuesday", "Thursday"]), # 25
    Professor("Dr. Öğr. Üyesi Sanam Moghaddamnia", preferred_days=["Tuesday", "Thursday"]), # 26
    Professor("Dr. Öğr. Üyesi Engin ARIKAN", preferred_days=["Tuesday", "Thursday"]), # 27
    Professor("Prof. Dr. Hüseyin Özcan", preferred_days=["Monday", "Tuesday", "Thursday"]), # 28
    Professor("Prof. Dr. Henning Rosenau", preferred_days=["Tuesday", "Thursday"]), # 29
    Professor("Prof. Dr. Katharina Gelbrich", preferred_days=["Tuesday", "Thursday"]), # 30
    Professor("Dr. Taner Aydin", preferred_days=["Tuesday", "Thursday"]), # 31
    Professor("Doç. Dr. M. Tolga Özer", preferred_days=["Tuesday", "Thursday"]), # 32
    Professor("Prof. Dr. Ali Kemal Yıldız", preferred_days=["Tuesday", "Thursday"]),  # 33
    Professor("Prof. Dr. Fatih Selami MAHMUTOĞLU", preferred_days=["Monday", "Tuesday", "Thursday"]), # 34
    Professor("Öğr. Gör. Dr. Ayşe Nur Saldıran", preferred_days=["Tuesday", "Thursday"]), # 35
    Professor("Dr. Susanne Schuster", preferred_days=["Tuesday", "Thursday"]), # 36
    Professor("Dr. Maria Liana Vodita", preferred_days=["Monday", "Tuesday", "Thursday"]), # 37
    Professor("Prof. Dr. Gerhard SEHER", preferred_days=["Tuesday", "Thursday"]), # 38
    Professor("Prof. Dr. Mesut Serdar Çekin", preferred_days=["Monday", "Wednesday", "Friday"]), # 39
    Professor("Prof. Dr. İrfan AKIN", preferred_days=["Monday", "Tuesday", "Thursday"]), # 40
    Professor("Prof. Dr. Hamide Özden Özkaya Ferendeci", preferred_days=["Tuesday", "Thursday"]), # 41
    Professor("Ralf Kanitz", preferred_days=["Monday", "Tuesday", "Thursday"]), # 42
    Professor("Dr. Taner Aydın", preferred_days=["Tuesday", "Thursday"]), # 43
    Professor("Doç. Dr. Üyesi Murat Sarıkaya", preferred_days=["Tuesday", "Thursday"]),  # 44
    Professor("Doç. Dr. Esra YİĞİT", preferred_days=["Tuesday", "Thursday"]), # 45
    Professor("Dr. Öğretim Üyesi Melis Avşar", preferred_days=["Monday", "Wednesday", "Friday"]), # 46
    Professor("Doç. Dr. Murat Sarıkaya", preferred_days=["Tuesday", "Thursday"]), # 47
    Professor("Doç. Dr. Berke Özenç", preferred_days=["Tuesday", "Thursday"]),  # 48
    Professor("Prof. Dr. Vural SEVEN", preferred_days=["Tuesday", "Thursday"]), # 49
    Professor("Dr. Esra Yılmaz Eren", preferred_days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), # 50
    Professor("Prof. Dr. Zafer Zeytin", preferred_days=["Tuesday", "Thursday"]), # 51
    Professor("Prof.Dr. Zafer Zeytin", preferred_days=["Tuesday", "Thursday"]), # 52
    Professor("Doç. Dr. Koray DEMİR", preferred_days=["Tuesday", "Thursday"]), # 53
    Professor("Dr. Hilal Ünal Kaya", preferred_days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), # 54
    Professor("Dr. Thorsten GERDES", preferred_days=["Tuesday", "Thursday"]), # 55
    Professor("Dr. Volkan Gezer (FF)", preferred_days=["Tuesday", "Thursday"]), # 56
    Professor("Prof. Dr. Barış Sertkaya (FF)", preferred_days=["Tuesday", "Thursday"]), # 57
    Professor("Prof. Dr. Mesut Güneş (FF)", preferred_days=["Monday", "Tuesday", "Friday"]), # 58
    Professor("Dr. Canan Yıldız", preferred_days=["Monday", "Tuesday", "Friday"]), # 59
    Professor("Dr. Emel Maden Yılmaz", preferred_days=["Tuesday", "Thursday"]), # 60
    Professor("Tobias Westphal", preferred_days=["Monday", "Wednesday", "Friday"]), # 61
    Professor("Dr. Burcu Yıldız", preferred_days=["Monday", "Tuesday", "Thursday"]), # 62
    Professor("Dr. Öğr. Üyesi Dilek Göksel Duru", preferred_days=["Monday", "Wednesday", "Friday"]), # 63
    Professor("Doç. Dr. Emre Işık", preferred_days=["Monday", "Tuesday", "Friday"]), # 64
    Professor("Dipl.-Ing. J. KUNTZE", preferred_days=["Tuesday", "Thursday"]), # 65
    Professor("Dr. Öğr. Üyesi Robin Kurilla", preferred_days=["Tuesday", "Thursday"]), # 66
    Professor("Prof. Dr. B. Aykut Arıkan", preferred_days=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), # 67
    Professor("Öğr. Gör. Esra JORKOWSKİ BERBEROĞLU", preferred_days=["Tuesday", "Thursday"]), # 68
    Professor("Dr.Öğr. Üyesi Büşra Fadim Sarıkaya", preferred_days=["Monday", "Tuesday", "Friday"]), # 69
    Professor("Prof. Uwe Hochmuth", preferred_days=["Monday", "Tuesday", "Thursday"]), # 70
    Professor("Araş.Gör. Dr. İpek Kesici", preferred_days=["Monday", "Tuesday", "Friday"]), # 71
]
classrooms = [
    Classroom("110", capacity=30, room_type="Labs"),
    Classroom("111", capacity=30, room_type="Labs"),
    Classroom("112", capacity=60, room_type="Labs"),
    Classroom("113", capacity=30, room_type="Labs"),
    Classroom("115-117", capacity=60, room_type="Labs"),
    Classroom("116", capacity=60, room_type="Labs"),
    Classroom("207-211", capacity=90, room_type="Labs"),
    Classroom("208", capacity=60, room_type="Labs"),
    Classroom("210-213", capacity=40, room_type="Labs"),
    Classroom("216-221", capacity=60, room_type="Labs"),
    Classroom("217", capacity=30, room_type="Labs"),
    Classroom("218", capacity=30, room_type="Labs"),
    Classroom("219", capacity=30, room_type="Labs"),
    Classroom("222-223", capacity=60, room_type="Labs"),
    Classroom("224", capacity=60, room_type="Labs"),
    Classroom("CS-Z-15", capacity=399, room_type="Vorlesung"),
    Classroom("EA-B1-12", capacity=130, room_type="Vorlesung"),
    Classroom("EA-B1-5", capacity=130, room_type="Vorlesung"),
    Classroom("ED-1-1", capacity=54, room_type="Vorlesung"),
    Classroom("ED-1-12", capacity=130, room_type="Vorlesung"),
    Classroom("ED-1-13", capacity=59, room_type="Vorlesung"),
    Classroom("ED-1-14", capacity=59, room_type="Vorlesung"),
    Classroom("ED-1-2", capacity=59, room_type="Vorlesung"),
    Classroom("ED-1-3", capacity=130, room_type="Vorlesung"),
    Classroom("ED-1-7", capacity=60, room_type="Vorlesung"),
    Classroom("ED-2-1", capacity=54, room_type="Vorlesung"),
    Classroom("ED-2-13", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-14", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-15", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-16", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-2", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-3", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-4", capacity=59, room_type="Vorlesung"),
    Classroom("ED-2-8", capacity=60, room_type="Vorlesung"),
    Classroom("ED-3-1", capacity=54, room_type="Vorlesung"),
    Classroom("ED-3-13", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-14", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-15", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-16", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-2", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-3", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-4", capacity=59, room_type="Vorlesung"),
    Classroom("ED-3-8", capacity=60, room_type="Vorlesung"),
    Classroom("ED-B1-1", capacity=80, room_type="Vorlesung"),
    Classroom("ED-B1-11", capacity=45, room_type="Vorlesung"),
    Classroom("ED-B1-12", capacity=60, room_type="Vorlesung"),
    Classroom("ED-B1-31", capacity=60, room_type="Vorlesung"),
    Classroom("ED-B1-33", capacity=80, room_type="Vorlesung"),
    Classroom("ED-B1-6", capacity=60, room_type="Vorlesung"),
    Classroom("ED-Z-1", capacity=54, room_type="Vorlesung"),
    Classroom("ED-Z-10", capacity=46, room_type="Vorlesung"),
    Classroom("ED-Z-12", capacity=130, room_type="Vorlesung"),
    Classroom("ED-Z-12-SB", capacity=46, room_type="Vorlesung"),
    Classroom("ED-Z-13", capacity=59, room_type="Vorlesung"),
    Classroom("ED-Z-13-SB", capacity=46, room_type="Vorlesung"),
    Classroom("ED-Z-14", capacity=59, room_type="Vorlesung"),
    Classroom("ED-Z-2", capacity=59, room_type="Vorlesung"),
    Classroom("ED-Z-20", capacity=40, room_type="Vorlesung"),
    Classroom("ED-Z-26", capacity=44, room_type="Vorlesung"),
    Classroom("ED-Z-27", capacity=50, room_type="Vorlesung"),
    Classroom("ED-Z-3", capacity=130, room_type="Vorlesung"),
    Classroom("ED-Z-38", capacity=46, room_type="Vorlesung"),
    Classroom("ED-Z-39", capacity=42, room_type="Vorlesung"),
    Classroom("ED-Z-5", capacity=56, room_type="Vorlesung"),
    Classroom("ED-Z-6", capacity=54, room_type="Vorlesung"),
    Classroom("ED-Z-7", capacity=60, room_type="Vorlesung"),
    Classroom("ED-Z-7-SB", capacity=76, room_type="Vorlesung"),
    Classroom("EL-B1-20", capacity=40, room_type="Labs"),
    Classroom("EL-B1-21", capacity=28, room_type="Labs"),
    Classroom("EL-B2-3", capacity=12, room_type="Labs"),
    Classroom("EL-B2-4", capacity=20, room_type="Labs"),
    Classroom("EL-B2-42", capacity=20, room_type="Labs"),
    Classroom("FMB-ES-103", capacity=20, room_type="Vorlesung"),
    Classroom("TDBA-EA-B1-12", capacity=70, room_type="Vorlesung"),
    Classroom("TDBA-EA-B1-5", capacity=70, room_type="Vorlesung"),
    Classroom("FAR-01", capacity=300, room_type="Vorlesung"),
    Classroom("Z1", capacity=120, room_type="Lab")
]
lessons = [
    Lesson(code="BAU091", teacher=professors[0], name="Einführung in das Bauingenieurwesen", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="BAU"),
    Lesson(code="BAU107", teacher=professors[1], name="Konstruktionslehre I: Technisches Zeichnen und CAD", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="BAU"),
    Lesson(code="BAU109", teacher=professors[2], name="Statik", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BAU"),
    Lesson(code="BAU201", teacher=professors[1], name="Bastoffe und Bauchemie I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BAU"),
    Lesson(code="BAU202", teacher=professors[0], name="Baustatik I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BAU"),
    Lesson(code="BAU301", teacher=professors[3], name="Konstruktiver Ingenieurbau II", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BAU"),
    Lesson(code="BAU302", teacher=professors[4], name="Grundbau und Bodenmechanik I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BAU"),
    Lesson(code="BAU303", teacher=professors[3], name="Verkehrwesen", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="BAU"),
    Lesson(code="BWL030", teacher=professors[5], name="Örgütsel Davranış", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="BWL"),
    Lesson(code="BWL033", teacher=professors[6], name="Personel Yönetimi", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL037", teacher=professors[7], name="İşletmesel Veri Analizi", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="BWL"),
    Lesson(code="BWL101", teacher=professors[8], name="İşletmeye Giriş", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BWL"),
    Lesson(code="BWL103", teacher=professors[9], name="Muhasebe", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="BWL"),
    Lesson(code="BWL105", teacher=professors[10], name="Statistik 1", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL109", teacher=professors[11], name="Matematik 1", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL201", teacher=professors[12], name="Pazarlamanın Temelleri", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL205", teacher=professors[9], name="Maliyet Muhasebesi", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BWL"),
    Lesson(code="BWL309", teacher=professors[6], name="Stratejik Yönetim", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL403", teacher=professors[13], name="İşletme Semineri", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="BWL"),
    Lesson(code="BWL413", teacher=professors[14], name="Pazar Araştırması", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="BWL"),
    Lesson(code="BWL415", teacher=professors[11], name="Yöneylem Araştırması", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BWL"),
    Lesson(code="BWL417", teacher=professors[15], name="İş Etiği", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="BWL"),
    Lesson(code="CHE111", teacher=professors[16], name="Kimya I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="CHE"),
    Lesson(code="DEU111", teacher=professors[17], name="İktisatçılar için Almanca 1", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="YDYO"),
    Lesson(code="DEU111", teacher=professors[17], name="İşletme Almancası 1", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="YDYO"),
    Lesson(code="DEU121", teacher=professors[17], name="Siyaset Bilimi için Almanca I (Deutsch für Politikwissenschaft I)", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="YDYO"),
    Lesson(code="DEU121", teacher=professors[17], name="Teknik Almanca I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="YDYO"),
    Lesson(code="DEU121", teacher=professors[17], name="Technisches Deutsch I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="YDYO"),
    Lesson(code="ENG101", teacher=professors[18], name="İngilizce 1", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="YDYO"),
    Lesson(code="ENG201", teacher=professors[18], name="İngilizce 3", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="YDYO"),
    Lesson(code="ENG203", teacher=professors[18], name="İngilizce 3", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="YDYO"),
    Lesson(code="ENG301", teacher=professors[18], name="İleri İngilizce 1", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="YDYO"),
    Lesson(code="ENG303", teacher=professors[18], name="İleri İngilizce 3", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="YDYO"),
    Lesson(code="ETE091", teacher=professors[19], name="Einführung in die Elektrotechnik", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="ETE"),
    Lesson(code="ETE101", teacher=professors[20], name="Digitaldesign", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="ETE"),
    Lesson(code="ETE201", teacher=professors[21], name="Elektrische Netzwerke I", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="ETE"),
    Lesson(code="ETE303", teacher=professors[22], name="Signale und Systeme", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="ETE"),
    Lesson(code="ETE311", teacher=professors[21], name="Elektronik I: Halbleiterbauelemente", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="ETE"),
    Lesson(code="ETE321", teacher=professors[23], name="Elektromagnetische Felder", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="ETE"),
    Lesson(code="ETE331", teacher=professors[24], name="Elektrische Maschinen I", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="ETE"),
    Lesson(code="ETE441", teacher=professors[25], name="FPGA-Programmierung", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="ETE"),
    Lesson(code="ETE459", teacher=professors[19], name="Nichtlineare Dynamische Systeme", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="ETE"),
    Lesson(code="ETE475", teacher=professors[26], name="Digitale Signalverarbeitung", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="ETE"),
    Lesson(code="HUK151", teacher=professors[27], name="Hukuka Giriş -Einführung in das Recht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK153", teacher=professors[28], name="Anayasa Hukuku I  (Genel Esaslar) -Verfassungsrecht I (Grundprinzipien)", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK155", teacher=professors[29], name="Ceza Hukukunun Temel Prensipleri -Grundprinzipien des Strafrechts", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="HUK"),
    Lesson(code="HUK157", teacher=professors[30], name="Hukuki İşlem Teorisi -Allgemeine Rechtsgeschäftslehre", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK159", teacher=professors[31], name="Anayasa Hukukunun Temel Prensipleri -Grundprinzipien des Verfassungsrechts", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK161", teacher=professors[32], name="Medeni Hukuka Giriş ve Başlangıç Hükümleri -Einführung in das Zivilrecht und Einleitungsartikel", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="HUK"),
    Lesson(code="HUK163", teacher=professors[33], name="Ceza Hukukuna Giriş -Einführung in das Strafrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK251", teacher=professors[34], name="Ceza Hukuku Genel Hükümler I -Strafrecht Allgemeiner Teil I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK253", teacher=professors[35], name="İdare Hukuku I Verwaltungsrecht I", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK255", teacher=professors[36], name="Milletlerarası Hukuk I -Völkerrecht I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK257", teacher=professors[37], name="Alman Sözleşme Hukuku -Deutsches Allgemeines Vertragsrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK259", teacher=professors[38], name="Hukuk Felsefesi ve Hukuk Sosyolojisi -Rechtsphilosophie und Rechtssoziologie", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK261", teacher=professors[39], name="Borçlar Hukuku Genel Hükümler I (Sözleşmeler Hukuku) -Schuldrecht Allgemeiner Teil I (Vertragsrecht)", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK351", teacher=professors[33], name="Ceza Hukuku Özel Hükümler I -Strafrecht Besonderer Teil I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK353", teacher=professors[40], name="Ticari İşletme Hukuku -Unternehmensrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK355", teacher=professors[32], name="Eşya Hukuku I (Zilyetlik ve Menkul Eşya Hukuku) -Sachenrecht I (Besitz und Mobiliarsachenrecht)", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK357", teacher=professors[41], name="Medeni Usul Hukuku I -Zivilprozessrecht I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK359", teacher=professors[42], name="Avrupa Hukuku -Europarecht", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK361", teacher=professors[43], name="Alman İdare Hukuku – Temel Kavramlar -Deutsches Verwaltungsrecht – Grundbegriffe", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="HUK"),
    Lesson(code="HUK363", teacher=professors[44], name="Özel Borç İlişkileri I -Schuldrecht Besonderer Teil I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK365", teacher=professors[45], name="İş ve Sosyal Güvenlik Hukuku I -Arbeits- und Sozialversicherungsrecht I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK451", teacher=professors[33], name="Ceza Muhakemesi Hukuku I -Strafprozessrecht I", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK453", teacher=professors[46], name="Devletler Özel Hukuku I (Vatandaşlık, Yabancılar ve Uluslararası Koruma Hukuku) -Internationales Privatrecht I (Staatsangehörigkeit, Recht der Ausländer und internationaler Schutz)", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK455", teacher=professors[47], name="Aile Hukuku -Familienrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK457", teacher=professors[41], name="İcra Hukuku -Zwangsvollstreckungsrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK459", teacher=professors[48], name="İnsan Hakları Hukuku -Menschenrechte", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK461", teacher=professors[43], name="Alman Özel İdare Hukuku -Deutsches Besonderes Verwaltungsrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK463", teacher=professors[49], name="Taşıma Hukuku -Transportrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK465", teacher=professors[40], name="Kıymetli Evrak Hukuku -Wertpapierrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK553", teacher=professors[40], name="Fikrî Mülkiyet Hukuku -Recht des geistigen Eigentums", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="HUK"),
    Lesson(code="HUK555", teacher=professors[50], name="Hukuk Kliniği -Rechtsklinik", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK557", teacher=professors[51], name="Tüketici Hukuku -Verbraucherrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="HUK"),
    Lesson(code="HUK753", teacher=professors[52], name="Milletlerarası Mal Satım Sözleşmeleri Hukuku -(CISG)UN-Kaufrecht (CISG)", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="HUK755", teacher=professors[53], name="Rekabet Hukuku -Wettbewerbsrecht", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="HUK"),
    Lesson(code="HUK915", teacher=professors[54], name="İş Hukuku", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="HUK"),
    Lesson(code="HUK931", teacher=professors[55], name="Hukuka Giriş", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="HUK"),
    Lesson(code="INF101", teacher=professors[56], name="Einführung in die Informatik und Programmierung", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="INF"),
    Lesson(code="INF103", teacher=professors[57], name="Logik für Informatiker", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="INF"),
    Lesson(code="INF107", teacher=professors[58], name="Rechnerorganisation", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="INF"),
    Lesson(code="INF201", teacher=professors[59], name="Diskrete Strukturen", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="INF"),
    Lesson(code="INF203", teacher=professors[60], name="Algorithmen und Datenstrukturen I; Algorithmen und Datenstrukturen I", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="INF"),
    Lesson(code="INF209", teacher=professors[58], name="Computernetze", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="INF"),
    Lesson(code="INF503", teacher=professors[59], name="Neural Networks (EN)", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="INF"),
    Lesson(code="INF506", teacher=professors[61], name="Methoden der Datenanalyse", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="INF"),
    Lesson(code="INF511", teacher=professors[62], name="Verteilte Systeme", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="INF"),
    Lesson(code="INF523", teacher=professors[63], name="Mensch-Maschine Kommunikation", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="INF"),
    Lesson(code="INF701", teacher=professors[59], name="Künstliche Intelligenz", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="INF"),
    Lesson(code="INF714", teacher=professors[64], name="Advanced Topics in Mathematics for Computer Scientists (EN)", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="INF"),
    Lesson(code="ISG001", teacher=professors[65], name="Arbeitsgesundheit und -sicherheit I", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="YDYO"),
    Lesson(code="KKW101", teacher=professors[66], name="Sosyal Bilimlerin Temelleri", is_online=False, class_type="Vorlesung", duration=2, capacity=45, section="KKW"),
    Lesson(code="KKW103", teacher=professors[67], name="İletişim Bilimlerine Giriş", is_online=False, class_type="Vorlesung", duration=2, capacity=20, section="KKW"),
    Lesson(code="KKW161", teacher=professors[68], name="Mesleki Almanca I: Kültür Bilimleri", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="KKW"),
    Lesson(code="KKW219", teacher=professors[69], name="Yeni Medya Teknolojileri", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="KKW"),
    Lesson(code="KKW233", teacher=professors[70], name="Medya Ve Kültür Bilimleri", is_online=False, class_type="Vorlesung", duration=2, capacity=30, section="KKW"),
    Lesson(code="KKW251", teacher=professors[71], name="İletişim Kuramları II", is_online=False, class_type="Vorlesung", duration=2, capacity=60, section="KKW")
]

days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
timeslots = ["9-10", "10-11", "11-12", "12-13", "13-14", "14-15", "15-16", "16-17", "17-18"]

In [20]:
def schedule_to_chromosome(schedule):
    chromosome = []
    for sl in schedule.scheduled_lessons:
        lesson_index = lessons.index(sl.lesson)
        professor_index = professors.index(sl.lesson.teacher)
        classroom_index = classrooms.index(sl.classroom)
        day_index = days_of_week.index(sl.day)
        timeslot_indexes = [timeslots.index(time_slot) for time_slot in sl.start_timeslot]

        chromosome.append([lesson_index, professor_index, classroom_index, day_index, timeslot_indexes])
    return chromosome

In [12]:
def create_random_schedule():
    scheduled_lessons = []
    for lesson in lessons:

        day = random.choice(lesson.preferred_days if lesson.preferred_days else days_of_week)

        possible_start_slots = [i for i in range(len(timeslots) - lesson.duration + 1)]
        start_slot_index = random.choice(possible_start_slots)
        assigned_timeslots = timeslots[start_slot_index:start_slot_index + lesson.duration]

        room_candidates = [room for room in classrooms if room.name in lesson.room_preference]
        classroom = random.choice(room_candidates if room_candidates else classrooms)

        scheduled_lessons.append(ScheduledLesson(lesson, classroom, day, assigned_timeslots))

    return Schedule(scheduled_lessons)

def create_population():
    population = [create_random_schedule() for _ in range(POPULATION_SIZE)]
    return population

# Create pop
population = create_population()

# Print the schedules in the initial population
for i, schedule in enumerate(population):
    print(f"Schedule {i+1}:")
    print(schedule)
    print("\n" + "="*20 + "\n")

Schedule 1:
[BAU091, Dr. Celal Çakıroğlu, 219, Friday, ['13-14', '14-15']]
[BAU107, Doç. Dr Enver Vural Yavuz, ED-2-15, Monday, ['9-10', '10-11']]
[BAU109, Dr. Ömer Faruk Aydın, ED-Z-5, Tuesday, ['13-14', '14-15']]
[BAU201, Doç. Dr Enver Vural Yavuz, 208, Thursday, ['9-10', '10-11']]
[BAU202, Dr. Celal Çakıroğlu, 219, Monday, ['16-17', '17-18']]
[BAU301, Dr. Serdar Ulusoy, EL-B2-4, Thursday, ['13-14', '14-15']]
[BAU302, Prof. Dr. F. RACKWITZ, 222-223, Wednesday, ['10-11', '11-12']]
[BAU303, Dr. Serdar Ulusoy, ED-1-13, Thursday, ['11-12', '12-13']]
[BWL030, Dr. Öğr. Üyesi Moritz BOTTS, 219, Friday, ['14-15', '15-16']]
[BWL033, Prof. Dr. Dilek ZAMANTILI NAYIR, ED-B1-1, Thursday, ['16-17', '17-18']]
[BWL037, M. Kerem Öztürk, 216-221, Friday, ['13-14', '14-15']]
[BWL101, Prof. Dr. Müge KLEIN, CS-Z-15, Wednesday, ['16-17', '17-18']]
[BWL103, Dr.Helmut Niesner, ED-Z-5, Tuesday, ['13-14', '14-15']]
[BWL105, Prof. Dr. Wolfgang KOHN, ED-2-4, Tuesday, ['16-17', '17-18']]
[BWL109, Dr. Öğr. Üyesi 

In [21]:
schedule_to_chromosome(population[0])

[[0, 0, 12, 4, [4, 5]],
 [1, 1, 28, 0, [0, 1]],
 [2, 2, 63, 1, [4, 5]],
 [3, 1, 7, 3, [0, 1]],
 [4, 0, 12, 0, [7, 8]],
 [5, 3, 70, 3, [4, 5]],
 [6, 4, 13, 2, [1, 2]],
 [7, 3, 20, 3, [2, 3]],
 [8, 5, 12, 4, [5, 6]],
 [9, 6, 43, 3, [7, 8]],
 [10, 7, 9, 4, [4, 5]],
 [11, 8, 15, 2, [7, 8]],
 [12, 9, 63, 1, [4, 5]],
 [13, 10, 32, 1, [7, 8]],
 [14, 11, 58, 1, [1, 2]],
 [15, 12, 8, 4, [1, 2]],
 [16, 9, 43, 2, [3, 4]],
 [17, 6, 4, 1, [1, 2]],
 [18, 13, 56, 1, [0, 1]],
 [19, 14, 13, 2, [5, 6]],
 [20, 11, 35, 3, [1, 2]],
 [21, 15, 59, 3, [4, 5]],
 [22, 16, 16, 2, [3, 4]],
 [23, 17, 15, 4, [0, 1]],
 [24, 17, 54, 2, [5, 6]],
 [25, 17, 42, 4, [5, 6]],
 [26, 17, 32, 2, [5, 6]],
 [27, 17, 12, 1, [4, 5]],
 [28, 18, 3, 3, [6, 7]],
 [29, 18, 38, 2, [2, 3]],
 [30, 18, 29, 4, [3, 4]],
 [31, 18, 65, 0, [6, 7]],
 [32, 18, 10, 4, [4, 5]],
 [33, 19, 26, 3, [7, 8]],
 [34, 20, 55, 0, [7, 8]],
 [35, 21, 54, 1, [7, 8]],
 [36, 22, 29, 2, [2, 3]],
 [37, 21, 25, 0, [6, 7]],
 [38, 23, 75, 0, [0, 1]],
 [39, 24, 47, 3,

In [ ]:
def roulette_wheel_selection(population, number_of_parents):
    total_fitness = sum(ind.fitness for ind in population)
    selected_parents = []

    for _ in range(number_of_parents):
        pick = random.uniform(0, total_fitness)
        current = 0

        for ind in population:
            current += ind.fitness
            if current > pick:
                selected_parents.append(ind)
                break

    return selected_parents

In [ ]:
def multipoint_crossover(parent1, parent2):
    scheduled_lessons1 = parent1.scheduled_lessons
    scheduled_lessons2 = parent2.scheduled_lessons

    crossover_points = sorted(random.sample(range(len(scheduled_lessons1)), 2))
    point1, point2 = crossover_points

    offspring1_lessons = scheduled_lessons1[:point1] + scheduled_lessons2[point1:point2] + scheduled_lessons1[point2:]
    offspring2_lessons = scheduled_lessons2[:point1] + scheduled_lessons1[point1:point2] + scheduled_lessons2[point2:]

    offspring1 = Schedule(offspring1_lessons)
    offspring2 = Schedule(offspring2_lessons)

    return offspring1, offspring2

In [ ]:
parents = roulette_wheel_selection(population, number_of_parents)
parent1,parent2 = parents

In [ ]:
offspring1,offspring2 = multipoint_crossover(parent1,parent2)

In [ ]:
print("Parent 1:")
print(parent1)
print("\n" + "="*20 + "\n")
print("Parent 2:")
print(parent2)
print("\n" + "="*20 + "\n")

Parent 1:
[INF101, Dr. Smith, C2, Wednesday, ['12-13', '13-14']]
[INF104, Prof. Johnson, C6, Monday, ['13-14', '14-15']]
[INF205, Prof. Yıldız, C3, Tuesday, ['10-11', '11-12']]
[INF204, Dr. Smith, C4, Thursday, ['16-17', '17-18']]
[INF304, Dr. Smith, C1, Monday, ['14-15', '15-16']]
[INF404, Dr. Smith, C4, Monday, ['11-12', '12-13']]
[INF504, Dr. Smith, C6, Tuesday, ['16-17', '17-18']]
[INF604, Dr. Smith, C6, Tuesday, ['14-15', '15-16']]
[INF704, Dr. Smith, C4, Thursday, ['15-16', '16-17']]
[INF303, Prof. Johnson, C6, Wednesday, ['11-12', '12-13']]


Parent 2:
[INF101, Dr. Smith, C5, Monday, ['9-10', '10-11']]
[INF104, Prof. Johnson, C6, Tuesday, ['11-12', '12-13']]
[INF205, Prof. Yıldız, C1, Tuesday, ['15-16', '16-17']]
[INF204, Dr. Smith, C4, Wednesday, ['9-10', '10-11']]
[INF304, Dr. Smith, C3, Thursday, ['10-11', '11-12']]
[INF404, Dr. Smith, C6, Tuesday, ['14-15', '15-16']]
[INF504, Dr. Smith, C2, Monday, ['12-13', '13-14']]
[INF604, Dr. Smith, C4, Thursday, ['13-14', '14-15']]
[IN

In [ ]:
print("Offspring 1:")
print(offspring1)
print("\n" + "="*20 + "\n")
print("Offspring 2:")
print(offspring2)
print("\n" + "="*20 + "\n")

Offspring 1:
[INF101, Dr. Smith, C2, Wednesday, ['12-13', '13-14']]
[INF104, Prof. Johnson, C6, Monday, ['13-14', '14-15']]
[INF205, Prof. Yıldız, C3, Tuesday, ['10-11', '11-12']]
[INF204, Dr. Smith, C4, Wednesday, ['9-10', '10-11']]
[INF304, Dr. Smith, C1, Monday, ['14-15', '15-16']]
[INF404, Dr. Smith, C4, Monday, ['11-12', '12-13']]
[INF504, Dr. Smith, C6, Tuesday, ['16-17', '17-18']]
[INF604, Dr. Smith, C6, Tuesday, ['14-15', '15-16']]
[INF704, Dr. Smith, C4, Thursday, ['15-16', '16-17']]
[INF303, Prof. Johnson, C6, Wednesday, ['11-12', '12-13']]


Offspring 2:
[INF101, Dr. Smith, C5, Monday, ['9-10', '10-11']]
[INF104, Prof. Johnson, C6, Tuesday, ['11-12', '12-13']]
[INF205, Prof. Yıldız, C1, Tuesday, ['15-16', '16-17']]
[INF204, Dr. Smith, C4, Thursday, ['16-17', '17-18']]
[INF304, Dr. Smith, C3, Thursday, ['10-11', '11-12']]
[INF404, Dr. Smith, C6, Tuesday, ['14-15', '15-16']]
[INF504, Dr. Smith, C2, Monday, ['12-13', '13-14']]
[INF604, Dr. Smith, C4, Thursday, ['13-14', '14-15'

In [ ]:
def mutate_individual(scheduled_lesson):

    if random.random() < MUTATION_RATE:
        dice = random.randrange(1, 4)
        if dice == 1:
            scheduled_lesson.classroom = random.choice(classrooms)
        elif dice == 2:
            scheduled_lesson.day = random.choice(days_of_week)
        elif dice == 3:
            possible_start_slots = [i for i in range(len(timeslots) - scheduled_lesson.lesson.duration + 1)]
            start_slot_index = random.choice(possible_start_slots)
            scheduled_lesson.start_timeslot = timeslots[start_slot_index:start_slot_index + scheduled_lesson.lesson.duration]

def mutation(offspring):
    for ind in offspring.scheduled_lessons:
        mutate_individual(ind)
    return offspring


In [ ]:
def elit_selection(population,offspring):
    combined_population = population + offspring

    new_population = sorted(combined_population, key=lambda ind: ind.fitness)[:POPULATION_SIZE]

    return new_population

In [ ]:
def evolutionary_algorithm():

    population = create_population()

    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")

        parents = roulette_wheel_selection(population, number_of_parents)

        offspring = []
        while len(offspring) < POPULATION_SIZE:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = multipoint_crossover(parent1, parent2)
            offspring.append(mutation(child1))
            if len(offspring) < POPULATION_SIZE:
                offspring.append(mutation(child2))

        population = elit_selection(population,offspring)

        best_individual = min(population, key=lambda x: x.fitness)
        print("Best individual in this generation:")
        print(best_individual)


evolutionary_algorithm()

Streaming output truncated to the last 5000 lines.
[INF205, Prof. Yıldız, C1, Friday, ['12-13', '13-14']]
[INF204, Dr. Smith, C6, Tuesday, ['10-11', '11-12']]
[INF304, Dr. Smith, C1, Tuesday, ['10-11', '11-12']]
[INF404, Dr. Smith, C1, Monday, ['10-11', '11-12']]
[INF504, Dr. Smith, C5, Tuesday, ['13-14', '14-15']]
[INF604, Dr. Smith, C5, Wednesday, ['14-15', '15-16']]
[INF704, Dr. Smith, C2, Friday, ['12-13', '13-14']]
[INF303, Prof. Johnson, C3, Tuesday, ['10-11', '11-12']]
Epoch 85/500
Best individual in this generation:
[INF101, Dr. Smith, C4, Thursday, ['9-10', '10-11']]
[INF104, Prof. Johnson, C6, Monday, ['16-17', '17-18']]
[INF205, Prof. Yıldız, C1, Friday, ['12-13', '13-14']]
[INF204, Dr. Smith, C6, Tuesday, ['10-11', '11-12']]
[INF304, Dr. Smith, C7, Tuesday, ['10-11', '11-12']]
[INF404, Dr. Smith, C1, Monday, ['10-11', '11-12']]
[INF504, Dr. Smith, C5, Tuesday, ['13-14', '14-15']]
[INF604, Dr. Smith, C5, Wednesday, ['14-15', '15-16']]
[INF704, Dr. Smith, C2, Friday, ['12-13'